In [1]:
import os, sys, inspect

import numpy as np
import pandas as pd


import matplotlib
from matplotlib import pyplot as plt

import collections
import time
import datetime

import re

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

working_dir = os.path.dirname(
    os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(working_dir)
sys.path.insert(0, parent_dir)

from scripts_python import scraper as scr

%load_ext autoreload
%autoreload 2

In [2]:
driver = scr.get_web_driver()

1. Chrome - Help - About Google Chrome
2. check version; e.g. `Version 79.0.3945.117 (Official Build) (64-bit)`
3. [download relevant ChromeDriver](https://chromedriver.chromium.org/downloads)

In [3]:
def load_glassdoor_page(
        driver: selenium.webdriver.chrome.webdriver.WebDriver,
        url: str = 'https://www.glassdoor.co.uk',
        ):

    print(url)
    driver.get(url)

In [4]:
load_glassdoor_page(driver)

https://www.glassdoor.co.uk


In [9]:
driver.quit()

In [5]:
element = driver.find_element_by_xpath(
    f"//boxy[@class='main flex flex-wide loggedOut lang-en en-GB gdGrid lockedHome _initOk noTouch desktop']"
    f"//div[@id='PageContent']"
    f"//div[@id='TopNav']"
    f"//div[@class='locked-home-lg-margins d-block pt-xsm px-std']"
    f"//div[@class='locked-home-sign-in']"
    )
element.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//boxy[@class='main flex flex-wide loggedOut lang-en en-GB gdGrid lockedHome _initOk noTouch desktop']//div[@id='PageContent']//div[@id='TopNav']//div[@class='locked-home-lg-margins d-block pt-xsm px-std']//div[@class='locked-home-sign-in']"}
  (Session info: chrome=79.0.3945.117)


In [ ]:
def sign_in_to_glassdoor(
        browser: selenium.webdriver.chrome.webdriver.WebDriver,
        ):
    try:
        element = browser.find_element_by_xpath(
            f"//boxy[@class='main flex flex-wide loggedOut lang-en en-GB gdGrid lockedHome _initOk noTouch desktop']"
            f"//div[@id='PageContent']"
            f"//div[@id='TopNav']"
            f"//div[@class='locked-home-lg-margins d-block pt-xsm px-std']"
            f"//div[@class='locked-home-sign-in']"
            )
        element.click()
    except NoSuchElementException:
        pass